# 1. Set up spark context and SparkSession

In [61]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


# 2. Load dataset

In [62]:
raw_data = spark.read.csv('InputFile.txt', inferSchema=True, sep='\t',header=True).toDF('words')

In [63]:
raw_data.count()

100000

In [64]:
raw_data.distinct().count()

93405

# 3.  Clean and Manipulation

In [65]:
u_raw_data= raw_data.distinct()

In [66]:
type(u_raw_data)

pyspark.sql.dataframe.DataFrame

In [67]:
u_raw_data.show(4, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ACTH Syndrome, Ectopic]  [Adrenal Gland Neoplasms]  [Adrenocorticotropic Hormone]  [Corticotropin-Releasing Hormone]  [Cushing Syndrome]  [Dexamethasone]  [Diagnosis, Differential]  [Humans]  [Hydrocortisone]                                  

In [68]:
## define udf function
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import *
def keywords_parser(s):
    s = s.replace(',', '')
    s1 = s.strip('[|]')
    s2 = re.split('\]\s+\[', s1)
    return [''.join(y.split()) for y in s2]

In [69]:
%time keywords_parser_udf = udf(keywords_parser, ArrayType(StringType()))

CPU times: user 1.79 ms, sys: 523 µs, total: 2.31 ms
Wall time: 11.6 ms


In [70]:
df = u_raw_data.select(keywords_parser_udf(u_raw_data.words)).toDF('term')

# 4. Build corpus and document-term matrix

In [71]:
from pyspark.ml.feature import CountVectorizer
#count_vectorizer = CountVectorizer(vocabSize=pow(2,4),inputCol='term', outputCol='features')
count_vectorizer = CountVectorizer(vocabSize=166,inputCol='term',outputCol='features')
countVectorizer_mod = count_vectorizer.fit(df)
countVectorizer_twitter = countVectorizer_mod.transform(df)

### 4.1 build corpus

In [72]:
voca = countVectorizer_mod.vocabulary
len(voca)

166

In [73]:
with open("voca.txt", "w") as f:
    for s in voca:
        f.write(str(s) +"\n")

### 4.2 build document-term matrix

In [74]:
df = countVectorizer_twitter.select('features')

In [75]:
df.count()

93405

In [76]:
df.show(truncate=False)

+-----------------------------------------------------------+
|features                                                   |
+-----------------------------------------------------------+
|(166,[0,26],[1.0,1.0])                                     |
|(166,[0,2,13,86,95,160],[1.0,1.0,1.0,1.0,1.0,1.0])         |
|(166,[1,22,24,115],[1.0,1.0,1.0,1.0])                      |
|(166,[0,28],[1.0,1.0])                                     |
|(166,[],[])                                                |
|(166,[0,3,4,44,152],[1.0,1.0,1.0,1.0,1.0])                 |
|(166,[0,2,3,4,5,9],[1.0,1.0,1.0,1.0,1.0,1.0])              |
|(166,[0,2,3,20,39],[1.0,1.0,1.0,1.0,1.0])                  |
|(166,[1,43,115],[1.0,1.0,1.0])                             |
|(166,[0,28,141],[1.0,1.0,1.0])                             |
|(166,[0,101],[1.0,1.0])                                    |
|(166,[1,18,39],[1.0,1.0,1.0])                              |
|(166,[0,80,84],[1.0,1.0,1.0])                              |
|(166,[0

In [77]:
type(df)

pyspark.sql.dataframe.DataFrame

In [78]:
from pyspark.ml.linalg import Vectors, DenseMatrix

In [79]:
dA = df.rdd.map(lambda vector: Vectors.dense(vector[0].toArray()))

In [80]:
type(dA)

pyspark.rdd.PipelinedRDD

In [81]:
dA.collect()

[DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 DenseVector([1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [53]:
from pyspark.mllib.linalg.distributed import *

def as_block_matrix(rdd, rowsPerBlock=93405, colsPerBlock=166):
    return IndexedRowMatrix(
        rdd.zipWithIndex().map(lambda xi: IndexedRow(xi[1], xi[0]))
    ).toBlockMatrix(rowsPerBlock, colsPerBlock)

In [54]:
matrixA = as_block_matrix(dA)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 2662, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-53-f3925d8264fa>", line 5, in <lambda>
  File "/opt/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/distributed.py", line 314, in __init__
  

# 6. Build term-term matrix

In [31]:
%time A = mat.transpose().dot(mat)

CPU times: user 20.4 ms, sys: 32 µs, total: 20.5 ms
Wall time: 19.8 ms


In [41]:
B = A.toarray()

In [56]:
type(B)

numpy.ndarray

In [44]:
B.shape

(166, 166)

# 7. Extract the frequency large than 50

In [72]:
import numpy as np
import pandas as pd
uptri = np.triu(B, 1)

In [73]:
uptri[uptri>50]

array([  5275.,  17556.,  14052., ...,     76.,     87.,     71.])

In [82]:
tempM = np.where( uptri > 50 )

In [122]:
index = pd.DataFrame(list(tempM)).transpose()

In [124]:
value = pd.DataFrame(list(uptri[uptri>50]))

In [140]:
vocab = pd.DataFrame(voca)
vocab.head()

,0
0,Humans
1,Animals
2,Female
3,Male
4,Adult


In [136]:
indexFreq = pd.concat([index,value],axis=1)
indexFreq.columns = ['index1','index2','freq']

In [138]:
indexFreq.head()

,index1,index2,freq
0,0,1,5275
1,0,2,17556
2,0,3,14052
3,0,4,10401
4,0,5,8189


In [153]:
join1 = indexFreq.join(vocab, on='index2',how='left')
join1.columns = ['index1','index2','freq','term.x']

In [154]:
join2 = join1.join(vocab, on='index1',how='left')
join2.columns = ['index1','index2','freq','term.x','term.y']

In [155]:
join2.head()

,index1,index2,freq,term.x,term.y
0,0,1,5275,Animals,Humans
1,0,2,17556,Female,Humans
2,0,3,14052,Male,Humans
3,0,4,10401,Adult,Humans
4,0,5,8189,MiddleAged,Humans


In [165]:
output = join2[['term.x','term.y','freq']]

In [163]:
output.head()

,term.x,term.y,freq
0,Animals,Humans,5275
1,Female,Humans,17556
2,Male,Humans,14052
3,Adult,Humans,10401
4,MiddleAged,Humans,8189
